<a href="https://colab.research.google.com/github/victorpena01/Analisis-Exploratorio/blob/main/Clase02_09_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio en clase 02.09.2023

## Estudiante: Victor Peña



# IMPORTACIÓN

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.linear_model import LinearRegression

# CREACIÓN DE DATAFRAME

In [32]:
np.random.seed(0)
X1 = np.random.rand(9)  # Valores aleatorios entre 0 y 1
X2 = np.random.rand(9)  # Valores aleatorios entre 0 y 1
X1 = X1*10
X2= X2*10


In [33]:
X1

array([5.48813504, 7.15189366, 6.02763376, 5.44883183, 4.23654799,
       6.45894113, 4.37587211, 8.91773001, 9.63662761])

In [34]:
X2

array([3.83441519, 7.91725038, 5.2889492 , 5.68044561, 9.25596638,
       0.71036058, 0.871293  , 0.20218397, 8.32619846])

In [35]:

y = X1 + X2**4

# Crear un DataFrame con las columnas x1, x2 e y
data = {'X1': X1, 'X2': X2, 'y': y}
df = pd.DataFrame(data)


print(df) # Mostrar el DataFrame


         X1        X2            y
0  5.488135  3.834415   221.658694
1  7.151894  7.917250  3936.292028
2  6.027634  5.288949   788.515446
3  5.448832  5.680446  1046.637958
4  4.236548  9.255966  7344.084687
5  6.458941  0.710361     6.713575
6  4.375872  0.871293     4.952183
7  8.917730  0.202184     8.919401
8  9.636628  8.326198  4815.672770


In [36]:
train = df.iloc[:6]
test = df.iloc[6:]

train_X = train.drop('y', axis=1)
train_y = train.y

test_X = test.drop('y', axis=1)
test_y = test.y


In [37]:
train_X

,X1,X2
0,5.488135,3.834415
1,7.151894,7.917250
2,6.027634,5.288949
3,5.448832,5.680446
4,4.236548,9.255966
5,6.458941,0.710361


In [38]:
test_X

,X1,X2
6,4.375872,0.871293
7,8.917730,0.202184
8,9.636628,8.326198


In [39]:
train_y

0     221.658694
1    3936.292028
2     788.515446
3    1046.637958
4    7344.084687
5       6.713575
Name: y, dtype: float64

In [40]:
test_y

6       4.952183
7       8.919401
8    4815.672770
Name: y, dtype: float64

# CHEQUEO DE LINEALIDAD

In [41]:
m1 = LinearRegression()
fit1 = m1.fit(train_X, train_y)
preds = fit1.predict(test_X)
print(f"\n{preds}")
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds))}\n")


[ -591.86753987 -3253.01723945  2603.80316671]
RMSE: 2301.3613050610593



In [43]:
train_X.X2 = (train_X.X2)**4

test_X.X2 = (test_X.X2)**4

print(test_X)

m2 = LinearRegression()

fit2 = m2.fit(train_X, train_y)

preds = fit2.predict(test_X)

print(f"\n{preds}")

print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds))}\n")

         X1           X2
6  4.375872     0.576311
7  8.917730     0.001671
8  9.636628  4806.036142

[   4.95218309    8.91940105 4815.67276977]
RMSE: 8.765542574431115e-13



# CUSTOM TRANSFORM

In [44]:
class ExperimentalTransformer(BaseEstimator, TransformerMixin):
  def __init__(self):
    print('\n>>>>>>>init() called.\n')

  def fit(self, X, y = None):
    print('\n>>>>>>>fit() called.\n')
    return self

  def transform(self, X, y = None):
    print('\n>>>>>>>transform() called.\n')
    X_ = X.copy() # creamos una copia para evitar cambios en el conjunto de datos original
    X_.X2 = (X_.X2)**4
    return X_

# REINICIO DE DATOS

In [46]:
train = df.iloc[:6]
test = df.iloc[6:]

train_X = train.drop('y', axis=1)
train_y = train.y

test_X = test.drop('y', axis=1)
test_y = test.y

# PIPELINE

In [47]:
print("create pipeline 1")
pipe1 = Pipeline(steps=[
                       ('linear_model', LinearRegression())
])

print("fit pipeline 1")
pipe1.fit(train_X, train_y)

print("predict via pipeline 1")
preds1 = pipe1.predict(test_X)

print(f"\n{preds1}")  # should be [13.72113586 16.93334467]
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds1))}\n")


create pipeline 1
fit pipeline 1
predict via pipeline 1

[ -591.86753987 -3253.01723945  2603.80316671]
RMSE: 2301.3613050610593



In [48]:
print("create pipeline 2")
pipe2 = Pipeline(steps=[
                       ('experimental_trans', ExperimentalTransformer()),    # esto dispara una llamda a __init__
                       ('linear_model', LinearRegression())
])

# una sintaxis alternativa y más corta para hacer lo anterior, sin nombrar cada paso, es:
#pipe2 = make_pipeline(ExperimentalTransformer(), LinearRegression())

print("fit pipeline 2")
pipe2.fit(train_X, train_y)

print("predict via pipeline 2")
preds2 = pipe2.predict(test_X)

print(f"\n{preds2}")  # should be [14. 17.]
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds2))}\n")

create pipeline 2

>>>>>>>init() called.

fit pipeline 2

>>>>>>>fit() called.


>>>>>>>transform() called.

predict via pipeline 2

>>>>>>>transform() called.


[   4.95218309    8.91940105 4815.67276977]
RMSE: 8.765542574431115e-13

